# Занятие 7

#### Лекторы: Ярослав Петрик, Полина Тихонова

*Составлено с использованием материалов курса лекций «Программирование на языке Python для сбора и анализа данных» Щурова И.В., НИУ ВШЭ*

## Извлечение данных из веб-страниц

### Загрузка веб-страницы: модуль `requests`

Если у вас не сработает строчка ниже, то сделайте `pip install requests` или `conda install requests` в командной строке (например, в *Anaconda Prompt*).

In [2]:
import requests

Модуль `requests` позволяет получать доступ к веб-страницам. Я не буду вдаваться в подробности протокола `http`, но надо понимать, что есть два распространенных способа доступа к веб-страницам: запрос типа *get* и типа *post* (хотя на самом деле видов http-запросов гораздо больше). Запрос типа get - это когда вы передаете серверу какую-то информацию в адресной строке. Например, если вы перейдете по такому адресу: https://www.google.ru/?q=python+анализ+данных, то этим вы просите гугл искать по запросу "python анализ данных вшэ". post-запрос - это когда вам нужно ввести информацию в какую-нибудь форму, например, ввести логин-пароль, который не будет отображать в адресной строке браузера.

Мы пока будем использовать get-запросы.

In [3]:
r = requests.get('http://www.miigaik.ru/')

Чтобы проверить, что страница нормально загрузилась есть команда

In [4]:
r.ok

True

Значение True говорит о том, что все прошло нормально.

In [5]:
q = requests.get('http://www.miigaik.ru/ajlfdjalsdjf')
print(q.ok)

False


Мы попытались перейти по несуществующей странице и она не загрузилась. Вернемся к успешному запросу `r`. Посмотрим на html исходник страницы командой

In [6]:
print(r.text)

    <!DOCTYPE html>
    <html>
    <head>
		<meta name="yandex-tableau-widget" content="logo=http://miigaik.ru/logoyandex.png, color=#ffffff" />
        <meta charset="utf-8">
        <title>Главная страница</title>
        <meta http-equiv="X-UA-Compatible" content="IE=edge">
		<meta name="yandex-verification" content="e1a9b84a8a958a63" />
		<meta name="sputnik-verification" content="6it3YueB5IsSk0Ze" /> 
        <link rel="icon" href="/bitrix/templates/modern_blue_s1/favicon.ico">
        <meta name="viewport" content="width=device-width, initial-scale=1">
        <link href="//fonts.googleapis.com/css?family=PT+Sans:400,700,400italic,700italic|PT+Serif+Caption:400,400italic&amp;subset=latin,cyrillic"
              rel="stylesheet">
        <script src="//ajax.googleapis.com/ajax/libs/jquery/2.1.3/jquery.min.js"></script>
        <link rel="stylesheet" type="text/css" href="//cdn.jsdelivr.net/gh/kenwheeler/slick@1.7.1/slick/slick.css"/>
        <!-- Add the slick-theme.css if you wan

### Немного про HTML

То, что вы видите выше — HTML-страница. HTML (HyperText Markup Language) — это такой язык разметки, являющийся частным случаем стандарта SGML. Другим частным случаем SGML является XML, с которым мы еще встретимся. 

Напишем простенькую HTML-страницу. Удобнее всего это делать в каком-либо редакторе. Но я запишу ее в файл через ноутбук.

In [40]:
my_html = '''
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset = "UTF-8">
    <title>Title</title>
</head>
<body>
<h1>Hello</h1>
<p>I'm a paragraph.</p>
<hr>
<ol>
    <li>One</li>
    <li>Two</li>
</ol>
    
</body>
</html>
'''

In [41]:
with open('my.html', 'w') as f:
    f.write(my_html)

Откройте `my.html` браузером и вы увидите простую веб-страничку. Видно что HTML разбит на специальные фрагменты, которые называются тегами. В тексте выше есть теги: `<html>`, `<head>`, `<title>` и т.д. Каждый тег отмечает какой-то кусочек веб-страницы. Тег `<title>` — это заголовок страницы. Тег `<ol>` отмечает упорядоченный список. Тег `<li>` отвечает элементу списка. Тег `<p>` — абзац (paragraph). Все перечисленные теги являются *парными*: они отмечают какой-то фрагмент текста (возможно, содержащий другие теги), помещая его между соответствующим открывающим и закрывающим тегом (например, `<li>` — открывающий тег, а `</li>` — закрывающий; всё, что между ними — это элемент списка). Исключением здесь является тег `<hr>`, который рисует горизонтальную линию (он работает и без `</hr>`).

Фактически HTML-страница представляет собой набор вложенных тегов. Можно сказать, что это дерево с корнем в теге `<html>`. У каждого тега есть потомки - те теги, которые непосредственно вложены в него. Например, у тега `<body>` потомками будут `<h1>`, `<p>`, `<hr>`, `<ol>`. Получается такое как бы генеалогическое древо.

HTML нас интересует с целью извлечения информации из такого дерева. Одним из наиболее популярных объектов для хранения информации являются таблицы, поэтмоу давайте вставим в наш файл небольшую таблицу: она обозначается тегом `<table>`, каждая строка таблицы выделяется тегом `<tr>` внутри `<table>`, а каждая ячейка — тегом `<td>` внутри `<tr>`.

In [19]:
my_html = '''
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset = "UTF-8">
    <title>Title</title>
    <style type='text/css;'>
        table {
        border-collapse: collapse;
    }

    table, th, td {
        border: 1px solid black;
    }
    </style>
</head>
<body>
<h1>Hello</h1>
<p>I'm a paragraph.</p>
<hr>
<ol>
    <li>One</li>
    <li>Two</li>
</ol>
<table>
    <tr>
        <td>
            Cell 1
        </td>
        <td>
            Cell 2
        </td>
    </tr>
    <tr>
        <td>
            Cell 3
        </td>
        <td>
            Cell 4
        </td>
    </tr>
</table>
</body>
</html>
'''
with open('my.html', 'w') as f:
    f.write(my_html)

Вот так выглядит эта страница:

![Вот такая страничка с табличкой](http://math-info.hse.ru/f/2015-16/all-py/my-html.png)

Допустим, что она лежит где-то на удалённом сайте. Давайте загрузим ее с помощью `requests` и попробуем извлечь какую-то информацию.

In [7]:
r = requests.get('http://math-info.hse.ru/f/2015-16/all-py/my.html')

## BeautifulSoup

Для обработки веб-страниц существует множество пакетов. Проблема с HTML в том, что большинство браузеров ведет себя «прощающе», и поэтому в вебе много плохо-написанных (не по стандарту HTML) HTML-страниц. Впрочем, обработка даже не вполне корректного HTML-кода не так сложна, если под рукой есть подходящие инструменты.

Мы будем пользоваться пакетом *Beautiful Soup 4*. Он входит в стандартную поставку *Anaconda*, но если вы используете другой дистрибутив Python, возможно, вам придётся его установить вручную с помощью `pip install beautifulsoup4`.

> Пакет под названием `BeautifulSoup` — скорее всего, не то, что вам нужно. Это третья версия (*Beautiful Soup 3*), а мы будем использовать четвертую. Так что нам нужен пакет `beautifulsoup4`. Чтобы было совсем весело, при импорте нужно указывать другое название пакета — `bs4`, а импортировать функцию под названием `BeautifulSoup`. В общем, сначала легко запутаться, но эти трудности нужно преодолеть однажды, а потом будет проще.

In [8]:
from bs4 import BeautifulSoup

Чтобы использовать *Beautiful Soup*, нужно передать функции `BeautifulSoup` текст веб-страницы (в виде одной строки). Чтобы он не ругался, я также вручную указываю название парсера (той программы, которая как раз и осуществляет обработку HTML) — с целью совместимости я использую `html.parser` (он входит в поставку Python и не требует установки), но вы можете также попробовать использовать `lxml`, если он у вас установлен.

In [9]:
page = BeautifulSoup(r.text, 'html.parser')

Что теперь лежит в переменной `page`? Давайте посмотрим.

In [23]:
page


<!DOCTYPE html>

<html lang="en">
<head>
<meta charset="utf-8">
<title>Title</title>
<style type="text/css;">
        table {
        border-collapse: collapse;
    }

    table, th, td {
        border: 1px solid black;
    }
    </style>
</meta></head>
<body>
<h1>Hello</h1>
<p>I'm a paragraph.</p>
<hr>
<ol>
<li>One</li>
<li>Two</li>
</ol>
<table>
<tr>
<td>
            Cell 1
        </td>
<td>
            Cell 2
        </td>
</tr>
<tr>
<td>
            Cell 3
        </td>
<td>
            Cell 4
        </td>
</tr>
</table>
</hr></body>
</html>

Мы видим, что объект `page` очень похож на строку, но, на самом деле, это не просто строка. К `page` можно делать запросы. Например:

In [10]:
page.html

<html lang="en">
<head>
<meta charset="utf-8"/>
<title>Title</title>
<style type="text/css;">
        table {
        border-collapse: collapse;
    }

    table, th, td {
        border: 1px solid black;
    }
    </style>
</head>
<body>
<h1>Hello</h1>
<p>I'm a paragraph.</p>
<hr/>
<ol>
<li>One</li>
<li>Two</li>
</ol>
<table>
<tr>
<td>
            Cell 1
        </td>
<td>
            Cell 2
        </td>
</tr>
<tr>
<td>
            Cell 3
        </td>
<td>
            Cell 4
        </td>
</tr>
</table>
</body>
</html>

Мы видим то, что внутри тега `<html>` (это почти вся страница, но самая первая строчка «отрезалась»). Можно пойти вглубь и посмотреть на содержимое `<head>`.

In [11]:
page.html.head

<head>
<meta charset="utf-8"/>
<title>Title</title>
<style type="text/css;">
        table {
        border-collapse: collapse;
    }

    table, th, td {
        border: 1px solid black;
    }
    </style>
</head>

Теперь мы видим только то, что внутри тега `<head>`. Мы можем пойти еще глубже, и получить то, что находится внутри тега `<title>`, который в свою очередь находится внутри тега `<head>` (говорят, что `<title>` является *потомком* `<head>`:

In [26]:
page.html.head.title

<title>Title</title>

Впрочем, можно было бы и не писать так подробно — поскольку в документе есть только один тег `<title>`, мы бы могли не указывать, что он находится внутри `<head>`, который находится внутри `<html>`.

In [27]:
page.head.title

<title>Title</title>

In [28]:
page.title

<title>Title</title>

Одним из потомков `<body>` является `<table>`. Ее можно получить вот так.

In [29]:
page.body.table

<table>
<tr>
<td>
            Cell 1
        </td>
<td>
            Cell 2
        </td>
</tr>
<tr>
<td>
            Cell 3
        </td>
<td>
            Cell 4
        </td>
</tr>
</table>

Допустим, что мне нужно получить несколько элементов с одинаковым тегом, например, все строки `<tr>`. Для этого используется такой синтаксис:

In [30]:
rows = page.body.table.findAll('tr')
rows

[<tr>
 <td>
             Cell 1
         </td>
 <td>
             Cell 2
         </td>
 </tr>, <tr>
 <td>
             Cell 3
         </td>
 <td>
             Cell 4
         </td>
 </tr>]

In [31]:
len(rows)

2

Мы видим, что это список из двух элементов. Так что по нему можно пройти циклом.

In [32]:
for i, row in enumerate(rows):
    print(i)
    print(row)

0
<tr>
<td>
            Cell 1
        </td>
<td>
            Cell 2
        </td>
</tr>
1
<tr>
<td>
            Cell 3
        </td>
<td>
            Cell 4
        </td>
</tr>


У нас есть 2 строчки и каждая из них является таким же объектом BeautifulSoup, как и все предыдущие. Так что к ним можно применить конструкцию row.td

In [33]:
for i, row in enumerate(rows):
    print(i)
    print(row.td)

0
<td>
            Cell 1
        </td>
1
<td>
            Cell 3
        </td>


Мы видим, что если внутри тега `<row>` есть несколько тегов `<td>`, то row.td возьмет первый из них. Поэтому мы получили первый столбец. Но нас интересует не сам тег `<td>`, а строка, которая там лежит. Её можно напечатать вот так.

In [34]:
for i, row in enumerate(rows):
    print(i)
    print(row.td.string)

0

            Cell 1
        
1

            Cell 3
        


Видно, что перед строкой идут ненужные пробелы. Удалим их командой strip

In [35]:
for i, row in enumerate(rows):
    print(i)
    print(row.td.string.strip())

0
Cell 1
1
Cell 3


Давайте загрузим таблицу в виде списка списков

In [36]:
table = []
for i, row in enumerate(rows):
    table.append([])
    for cell in row.findAll('td'):
        table[-1].append(cell.string.strip())
print(table)

[['Cell 1', 'Cell 2'], ['Cell 3', 'Cell 4']]


Вот то же самое, но короче с помощью list comprehensions:

In [37]:
table = []
for row in rows:
    table.append([cell.string.strip() for cell in row.findAll('td')]) 
print(table)

[['Cell 1', 'Cell 2'], ['Cell 3', 'Cell 4']]


Или еще короче (но заковыристее):

In [38]:
table = [[cell.string.strip() for cell in row.findAll('td')] 
         for row in rows]
print(table)

[['Cell 1', 'Cell 2'], ['Cell 3', 'Cell 4']]


Заметим, что вместо `some_beautiful_soup_objec.findAll('sometag')` можно писать короче `some_beautiful_soup_object('sometag')`. Так что можно написать еще короче

In [39]:
table = [[cell.string.strip() for cell in row('td')] 
         for row in rows]
print(table)

[['Cell 1', 'Cell 2'], ['Cell 3', 'Cell 4']]


У тегов, кроме названия, бывают еще свойства — например, в строчке `<html lang="en">` мы видим свойство `lang` у тега `<html>`, имеющее значение `"en"`. Другим важным примером тега со свойствами является тег `<a>`, который создает ссылку. У него есть свойство `href`, которое хранит собственно ссылку.

> Например, строка
> `<a href="http://math-info.hse.ru/s15/m">Курс по Python</a>`
> превращается в ссылку
> <a href="http://math-info.hse.ru/s15/m">Курс по Python</a>,
> ведущую на страницу нашего курса.

Теперь представим себе, что мы хотим сделать робота, который будет ходить по веб-страницам, и переходить с одной страницы на другую по ссылкам. Тогда мы сталкиваемся с задачей извлечь из страницы все гиперссылки.
Для этого нужно найти все теги `<a>` на странице, и у всех них взять параметр `<href>`. Для начала покажем как получить свойство объекта, например, `lang` у `html`. Это делается так как будто наш объект словарь, и мы берем его значение по ключу.

In [13]:
page.html['lang']

'en'

Если запросить свойство, которое тег не имеет, то мы получим KeyError, как и со словарем.

In [12]:
page.html['strange']

KeyError: 'strange'

Так же, как у словаря, есть метод `get()`, который ничего не возвращает, если такого свойства нет. Или возвращает значение по умолчанию, определенное нами.

In [105]:
page.html.get('strange')

In [106]:
page.html.get('strange', 'no-such-tag')

'no-such-tag'

Теперь извлечем все ссылки с какого-нибудь сайта

In [108]:
r = requests.get('http://vyshka.math.ru')
page = BeautifulSoup(r.text, 'html.parser')

Вот все ссылки на нашей странице.

In [109]:
page('a')

[<a href="http://top100.rambler.ru/top100/">
 <img alt="" border="0" height="1" src="http://counter.rambler.ru/top100.cnt?188597" width="1"/></a>,
 <a href="http://math.hse.ru/">ÏÆÉÃÉÁÌØÎÏÍÕ ÓÁÊÔÕ 
       ÆÁËÕÌØÔÅÔÁ ÍÁÔÅÍÁÔÉËÉ</a>,
 <a href="http://hse.ru">îéõ ÷ûü</a>,
 <a href="1516/topology2.php">÷×ÅÄÅÎÉÅ × ÔÏÐÏÌÏÇÉÀ</a>,
 <a href="1516/algebra2.php">áÌÇÅÂÒÁ</a>,
 <a href="1516/funcan.php">æÕÎËÃÉÏÎÁÌØÎÙÊ ÁÎÁÌÉÚ</a>,
 <a href="1516/DiffGeom.php">äÉÆÆÅÒÅÎÃÉÁÌØÎÁÑ ÇÅÏÍÅÔÒÉÑ</a>,
 <a href="1516/tvs.php">ôÏÐÏÌÏÇÉÞÅÓËÉÅ ×ÅËÔÏÒÎÙÅ ÐÒÏÓÔÒÁÎÓÔ×Á</a>,
 <a href="1516/basic.php">ïÓÎÏ×ÎÙÅ ÐÏÎÑÔÉÑ ÍÁÔÅÍÁÔÉËÉ</a>,
 <a href="0809/0809.html">2008/09</a>,
 <a href="0910/0910.html">2009/10</a>,
 <a href="1011/1011.html">2010/11</a>,
 <a href="1112/1112.html">2011/12</a>,
 <a href="1213/1213.php">2012/13</a>,
 <a href="1415/1415.php">2014/15</a>]

Как видим, метод `findAll()` (или его сокращённая форма записи в виде просто скобочек) ищет не только по непосредственным «детям» какой-то вершины (в генеалогических терминах), но и по всем потомкам.

Напечатаем сами ссылки

In [110]:
for link in page("a"):
    print(link['href'])

http://top100.rambler.ru/top100/
http://math.hse.ru/
http://hse.ru
1516/topology2.php
1516/algebra2.php
1516/funcan.php
1516/DiffGeom.php
1516/tvs.php
1516/basic.php
0809/0809.html
0910/0910.html
1011/1011.html
1112/1112.html
1213/1213.php
1415/1415.php




## P.S. Документация — ваш друг
 Для *Beautiful Soup* документация лежит [здесь](http://www.crummy.com/software/BeautifulSoup/bs4/doc/), а для *requests* [здесь](http://docs.python-requests.org/en/latest/) (начните с Quickstart). 

Другой источник информации о библиотеках — всё тот же веб-поиск, который чаще всего будет выдавать ссылки на сайт с вопросами и ответами http://stackoverflow.com/. Например, набрав [how to parse table with beautifulsoup](https://www.google.ru/search?q=how+to+parse+table+with+beautifulsoup&gws_rd=cr&ei=wXaJVvzQKIfXyQO4v4PYDw) вы получите несколько ссылок на stackoverflow с примерами кода. Кстати, на stackoverflow можно задавать и свои вопросы — но прежде нужно убедиться, что на них не ответили раньше.

## Geopandas

In [18]:
import geopandas
import pandas as pd
pd.options.display.max_rows = 4

ModuleNotFoundError: No module named 'geopandas'